In [1]:
import pandas as pd
import numpy as np
import csv

df = pd.read_csv('nasdaq/AppleInc.csv', index_col = 'Date', parse_dates=True)
iter_len = len(df['Close'])


# Labeling: increase = 1, decrease = -1

with open('Apple.csv', 'w', newline='') as f:
    wt = csv.writer(f)
    wt.writerow(['avg'])
    #wt.writerow("Date,Open,High,Low,Close,Adj Close,Volume,Label".split(","))
    for i in range(iter_len):
        line1 = df.iloc[i]
        a1 = (line1['Open']+line1['High']+line1['Low']+line1['Close'])/4
        wt.writerow([a1])

In [9]:
df = pd.read_csv('Apple.csv')['avg']
iter_len = len(df)


with open('Apple-1.csv', 'w', newline='') as f:
    wt = csv.writer(f)
    wt.writerow("1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,label".split(","))
    for i in range(iter_len-21):
        line1 = [df.iloc[j] for j in range(i,i+20)]
        M=max(line1)
        m=min(line1)
        new_line = [(x-m)/(M-m) for x in line1]
        if ( df.iloc[i+19] < df.iloc[i+20] ):
            new_line.append(1)
        else: new_line.append(-1)
        wt.writerow(new_line)


In [10]:
df = pd.read_csv('Apple-1.csv')
print(df)

              1         2         3         4         5         6         7  \
0      0.323616  0.186598  0.000000  0.055390  0.125378  0.276971  0.405249   
1      0.186598  0.000000  0.055390  0.125378  0.276971  0.405249  0.521875   
2      0.000000  0.055390  0.125378  0.276971  0.405249  0.521875  0.673469   
3      0.000000  0.074091  0.234574  0.370374  0.493839  0.654322  0.950621   
4      0.000000  0.173325  0.319991  0.453335  0.626661  0.946669  1.000000   
...         ...       ...       ...       ...       ...       ...       ...   
10282  0.720465  0.703338  0.558491  0.296662  0.301887  0.274021  0.150943   
10283  0.703338  0.558491  0.296662  0.301887  0.274021  0.150943  0.000000   
10284  0.537130  0.285316  0.290340  0.263540  0.145170  0.000000  0.022892   
10285  0.285316  0.290340  0.263540  0.145170  0.000000  0.022892  0.017029   
10286  0.280172  0.254310  0.140086  0.000000  0.022091  0.016433  0.320851   

              8         9        10  ...        12 

In [11]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F



In [13]:
x=np.array(X)
y=np.array(y)

if torch.cuda.is_available():
    x=torch.tensor(x).cuda()
    y=torch.tensor(y).cuda()
    print("CUDA available")
else:
    x=torch.tensor(x)
    y=torch.tensor(y)

In [19]:
class Net(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.l1 = nn.Linear(D_in, H)
        self.h1 = nn.Linear(H, 2*H)
        self.h2 = nn.Linear(2*H, 4*H)
        self.l2 = nn.Linear(4*H, D_out)

    def forward(self, x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.h1(x))
        x = F.leaky_relu(self.h2(x))
        y = self.l2(x)
        return y


D_in, D_out = 20, 1 #input, output 

def run(H, lr, epochs):
  
    #training
    
    model = Net(D_in, H, D_out)
    if torch.cuda.is_available(): model = model.cuda()
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for t in range(epochs):
        y_pred = model(x.float())
        loss = criterion(y_pred, y.float())
        print(t, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  ###############################################################################
  
    #evaluation
    total=0
    correct=0
    
    for idx, inp in enumerate(x):
        total+=1
        y_pred = list(model(inp.float()).cpu().detach().numpy())
        y_truth = list(y[idx])
        print(y_pred, y_truth)
        #if y_pred.index(max(y_pred))==y_truth.index(max(y_truth)): correct+=1
    print(round(100*correct/total,2))


In [20]:
run(10, 1e-3, 10)

0 106126584.0
1 2.6477040283141288e+32
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan


TypeError: iteration over a 0-d tensor